In [1]:
#http://www.ritchieng.com/machine-learning/deep-learning/tensorflow/deep-neural-nets/
#http://ischlag.github.io/2016/06/04/how-to-use-tensorboard/
#https://jhui.github.io/2017/03/12/TensorBoard-visualize-your-learning/
#https://thecodacus.com/tensorboard-tutorial-visualize-networks-graphically/#.WlVvmnWnG00

from pandas import set_option
from pandas import read_csv
from numpy import array
from numpy import float32
import tensorflow as tf

In [2]:
tf.set_random_seed(42)

In [3]:
set_option('display.max_colwidth', -1)
set_option('precision', 18)

In [4]:
es = read_csv("../vectors/es.node2vec.embeddings", delimiter = " ", skiprows = 1, header = None)
na = read_csv("../vectors/na.node2vec.embeddings", delimiter = " ", skiprows = 1, header = None)

In [5]:
es.head()

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
0,igual,-0.021394000000000000,-0.223086000000000007,0.081375000000000003,-0.440008000000000010,0.414202999999999988,-0.377524000000000026,-0.405760999999999983,-0.080980999999999984,-0.143192000000000014,...,-0.185254000000000002,0.175382000000000010,0.326658000000000004,0.094920000000000004,-0.696819999999999995,0.002694000000000000,-0.451108999999999982,-0.471277999999999975,0.041342999999999998,-0.385863000000000012
1,hermano,-0.275133000000000016,-0.290683999999999998,-0.035325000000000002,-0.172819000000000000,-0.171268000000000004,0.194177999999999989,-0.260794999999999999,-0.131469000000000003,-0.560057000000000027,...,-0.772757000000000027,0.263089999999999991,0.152055999999999997,-0.348088999999999982,0.317742000000000024,0.093362000000000001,-0.539200000000000013,-0.091987000000000013,0.245604999999999990,0.074918999999999999
2,martajan,-0.390027000000000013,-0.181653000000000009,-0.266087999999999991,0.134891000000000011,-0.105632999999999991,-0.124488000000000015,-0.857403000000000026,-0.210448999999999969,0.100710999999999995,...,-0.305485000000000007,-0.438153999999999988,0.633051999999999948,-0.086624000000000007,0.049135999999999999,-0.166021000000000002,-0.458888999999999991,0.214303999999999994,-0.089515999999999998,-0.083751000000000006
3,fuero,0.038670999999999997,-0.126187999999999995,-0.128566999999999987,-0.107231000000000007,0.091522000000000006,-0.151676000000000005,-0.137239000000000000,-0.118835999999999983,-0.078329999999999983,...,-0.221096999999999988,0.062376000000000001,0.249082000000000026,0.034577999999999998,-0.069553000000000004,0.115556000000000006,-0.058006000000000002,0.075348999999999999,-0.039742000000000000,0.043875999999999998
4,ehecatl,-0.262041999999999997,-0.341544000000000014,0.020990999999999999,-0.098941000000000001,0.006511000000000000,-0.406260999999999983,-0.011587000000000000,-0.046593999999999997,0.200022000000000005,...,-0.087499999999999981,-0.129405999999999993,-0.203127000000000002,-0.141165000000000013,-0.115330000000000002,-0.204074000000000005,-0.148682000000000009,-0.107581999999999997,-0.051375999999999998,0.160946000000000006


In [6]:
na.head()

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
0,tihuitz,-0.348621000000000014,-0.017441000000000002,-0.155883999999999995,-0.418601000000000001,0.666802000000000006,-0.385216000000000003,-0.507603000000000026,0.115668999999999994,-0.210249999999999992,...,-0.577107000000000037,-0.059977999999999997,0.339285000000000003,0.037353999999999998,-0.344376000000000015,0.141060999999999992,0.233917999999999987,-0.402866000000000002,-0.363315999999999972,0.590776999999999997
1,quixtia,0.163007999999999986,-0.278059999999999974,-0.453527000000000013,-0.064904000000000003,0.025853999999999999,-0.084598000000000007,-0.170526000000000011,-0.208350000000000007,-0.172448999999999991,...,-0.529113999999999973,0.010237000000000001,0.433474000000000026,0.337760000000000005,-0.287584000000000006,0.015219999999999999,-0.413629000000000024,-0.150796000000000013,-0.156616000000000005,-0.005837000000000000
2,quixtiz,-0.689479999999999982,-0.714752999999999972,-0.100995000000000001,-0.094793000000000002,1.021972000000000103,0.090782000000000002,-0.129735999999999990,0.228915000000000007,0.007809000000000000,...,-0.114314000000000013,-0.053691999999999997,-0.343855000000000022,-0.012789999999999999,0.168138000000000010,0.322913999999999979,0.185893000000000003,0.157933999999999991,-0.369580999999999993,0.330840999999999996
3,triarcas,-0.239407000000000009,0.284032999999999980,0.120584999999999998,-0.452631999999999979,-0.201202999999999993,-0.211770999999999987,-0.927865999999999969,-0.209665999999999991,0.436325000000000018,...,-1.114122000000000057,-0.185642000000000001,-0.066564999999999999,0.003861000000000000,0.115815000000000015,-0.495630999999999988,0.358493000000000006,0.265313000000000021,-0.835150999999999977,-0.235812999999999995
4,peuh,-0.181800999999999990,-0.273990999999999985,-0.298422999999999994,0.112453999999999998,-0.174767000000000006,0.226840999999999987,-0.200030000000000013,-0.225804000000000005,-0.098054000000000002,...,-0.059520000000000003,-0.505542000000000047,0.070814000000000002,0.318340999999999985,-0.279799999999999993,0.314554000000000000,-0.474447999999999981,0.066695000000000004,-0.559996000000000049,-0.110311999999999993


In [7]:
es_vectores = array(es[:1900].loc[:,1::]).astype(float32)
na_vectores = array(na[:1900].loc[:,1::]).astype(float32)

In [8]:
es_vectores

array([[ -2.13939995e-02,  -2.23085999e-01,   8.13750029e-02, ...,
         -4.71278012e-01,   4.13429998e-02,  -3.85863006e-01],
       [ -2.75133014e-01,  -2.90684015e-01,  -3.53249982e-02, ...,
         -9.19869989e-02,   2.45605007e-01,   7.49190003e-02],
       [ -3.90026987e-01,  -1.81652993e-01,  -2.66088009e-01, ...,
          2.14304000e-01,  -8.95159990e-02,  -8.37510005e-02],
       ..., 
       [ -3.08957011e-01,  -7.55291998e-01,  -2.55849987e-01, ...,
         -1.68337002e-01,  -2.70969987e-01,  -1.13678001e-01],
       [ -1.12017997e-01,  -3.38728994e-01,  -5.40111005e-01, ...,
          1.00536004e-01,  -4.34828013e-01,   1.95170999e-01],
       [  1.72402993e-01,  -1.33572996e-01,  -5.93950003e-02, ...,
          4.24019992e-01,  -5.63090980e-01,   5.63999987e-04]], dtype=float32)

In [9]:
na_vectores

array([[-0.34862101, -0.017441  , -0.155884  , ..., -0.40286601,
        -0.363316  ,  0.59077698],
       [ 0.163008  , -0.27805999, -0.453527  , ..., -0.150796  ,
        -0.156616  , -0.005837  ],
       [-0.68948001, -0.71475297, -0.100995  , ...,  0.157934  ,
        -0.36958101,  0.330841  ],
       ..., 
       [-0.60704499,  0.019681  , -0.16201   , ..., -0.695319  ,
        -0.28995901, -0.67276502],
       [-0.086007  , -0.506019  , -0.02504   , ...,  0.053332  ,
         0.200712  , -0.14809801],
       [-0.242456  , -0.40662399, -0.44837299, ...,  0.204762  ,
         0.063915  ,  0.040679  ]], dtype=float32)

In [10]:
es_palabras = es[0]
na_palabras = na[0]

In [11]:
print("shape es:",es_vectores.shape,"\nshape na:",na_vectores.shape)

shape es: (1900, 128) 
shape na: (1900, 128)


# Hyperparameters

In [12]:
# Hyperparameters
LEARNING_RATE = 0.1
EPOCHS = 10
BATCH_SIZE = 100

NODES_INPUT = es_vectores[0].size
NODES_H1 = 128
NODES_H2 = 128
NODES_OUPUT = na_vectores[0].size
INSTANCES = es_vectores.__len__()
NUM_STEPS = 5000

In [13]:
X = tf.placeholder(shape=[None,NODES_INPUT], dtype=tf.float32,name='input_es')
y = tf.placeholder(shape=[None,NODES_OUPUT], dtype=tf.float32,name='target_na')
print("X:",X.shape,"y:",y.shape)

X: (?, 128) y: (?, 128)


In [14]:
# Capas ocultas (weights & bias)
hidden_layer1 = {"W1":tf.Variable(tf.truncated_normal([NODES_INPUT,NODES_H1],stddev=0.01),name = 'W1'),
                 "b1":tf.constant(0.1,shape=[NODES_H1], name = 'b1')}

hidden_layer2 = {"W2":tf.Variable(tf.truncated_normal([NODES_H1,NODES_H2],stddev=0.01),name = 'W2'),
                 "b2":tf.constant(0.1,shape=[NODES_H2], name = 'b2')}

# Capa de salida
output_layer = {"W_out":tf.Variable(tf.truncated_normal([NODES_H1,NODES_OUPUT],stddev=0.01),
                name = 'W_output'),
                "b_out":tf.constant(0.1,shape=[NODES_OUPUT], name = 'b_output')}


tf.summary.histogram("weight_1", hidden_layer1["W1"])
tf.summary.histogram("weight_2", hidden_layer2["W2"])
tf.summary.histogram("weight_out", output_layer["W_out"])

<tf.Tensor 'weight_out:0' shape=() dtype=string>

In [15]:
print(hidden_layer1)

{'W1': <tf.Variable 'W1:0' shape=(128, 128) dtype=float32_ref>, 'b1': <tf.Tensor 'b1:0' shape=(128,) dtype=float32>}


In [16]:
# Calcular la salida de la 1er capa oculta
# h(x) = x*w + bias
hidden_layer1_output = tf.add(tf.matmul(es_vectores, hidden_layer1["W1"]), hidden_layer1["b1"])
# Función de activación usando ReLU
tf.summary.histogram("pre_activations_h1", hidden_layer1_output)
hidden_layer1_output = tf.nn.relu(hidden_layer1_output,name="h1Activation")
tf.summary.histogram('activationsh1', hidden_layer1_output)

<tf.Tensor 'activationsh1:0' shape=() dtype=string>

In [17]:
# Calcular la salida de la 2da capa oculta
# h(x) = x*w + bias
hidden_layer2_output = tf.add(tf.matmul(hidden_layer1_output, hidden_layer2["W2"]), hidden_layer2["b2"])
# Función de activación usando ReLU
tf.summary.histogram("pre_activations_h2", hidden_layer2_output)
hidden_layer2_output = tf.nn.relu(hidden_layer2_output,name="h2Activation")
tf.summary.histogram('activationsh2', hidden_layer2_output)

<tf.Tensor 'activationsh2:0' shape=() dtype=string>

In [18]:
# calcular la salida la NN
output = tf.add(tf.matmul(hidden_layer2_output, output_layer["W_out"]),output_layer["b_out"])
# Función de activación usando softmax
tf.summary.histogram("pre_activations_output", output)
nah_predicted = tf.nn.softmax(output,name="outActivation")
tf.summary.histogram('activationsout', nah_predicted)
print(nah_predicted.shape,y.shape)

(1900, 128) (?, 128)


In [19]:
# Función de error (Mean Square Error)
#loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=nah_predicted))
loss = tf.reduce_mean(tf.squared_difference(nah_predicted,y),name="loss_f")
#loss = tf.reduce_mean(tf.reduce_sum((nah_predicted - y) ** 2))
tf.summary.scalar("cost",loss)

<tf.Tensor 'cost:0' shape=() dtype=string>

In [20]:
# optimiser, 
optimiser = tf.train.AdagradOptimizer(learning_rate=LEARNING_RATE,
                                      name="AdagradOptimizer").minimize(loss, name="loss")

In [21]:
#Session
sess = tf.Session()
# Initialize variables
summaryMerged = tf.summary.merge_all()
writer = tf.summary.FileWriter("./logs/NN",sess.graph)
run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
run_metadata = tf.RunMetadata()
init = tf.global_variables_initializer()
sess.run(init)

In [22]:
tmp_hidden_layer1,tmp_hidden_layer2 = sess.run(hidden_layer1),sess.run(hidden_layer2)

In [23]:
for i in range(3000):
    '''
    offset = (step * BATCH_SIZE) % (es_vectores.shape[0] - BATCH_SIZE)
    batch_data = es_vectores[offset:(offset + BATCH_SIZE), :]
    batch_target = na_vectores[offset:(offset + BATCH_SIZE), :]
    '''
    _loss,_, sumOut = sess.run([loss,optimiser, summaryMerged], feed_dict={
                        X: es_vectores, y: na_vectores},
                        options=run_options,
                        run_metadata=run_metadata)
    if (i % 100) == 0:
        print(_loss)
    writer.add_summary(sumOut, i)
    writer.add_run_metadata(run_metadata, 'step%03d' % i)
writer.close()

0.159136
0.159132
0.159129
0.159126
0.159123
0.15912
0.159116
0.159113
0.15911
0.159107
0.159103
0.1591
0.159096
0.159093
0.15909
0.159086
0.159083
0.15908
0.159076
0.159073
0.159069
0.159066
0.159062
0.159059
0.159055
0.159051
0.159048
0.159044
0.15904
0.159036


In [24]:
print(sess.run(hidden_layer1))

{'W1': array([[ 0.01083493, -0.01566196, -0.00410667, ..., -0.00440193,
         0.00182029, -0.00193312],
       [ 0.01465998, -0.0098182 ,  0.00280799, ...,  0.00754412,
        -0.00551736,  0.009543  ],
       [ 0.01194355,  0.00044788, -0.00274036, ...,  0.01057103,
        -0.01332162,  0.00339089],
       ..., 
       [-0.00548137,  0.00695821, -0.00708327, ...,  0.00341065,
        -0.01197848, -0.00558517],
       [-0.00745449, -0.00052819, -0.00955121, ..., -0.00074251,
        -0.0143069 ,  0.00596605],
       [-0.00782242,  0.00816668, -0.00558511, ...,  0.00087154,
         0.0056173 , -0.00862032]], dtype=float32), 'b1': array([ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,
        0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,
        0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,
        0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,
        0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1

In [27]:

tmp_hidden_layer1["W1"]-sess.run(hidden_layer1["W1"])

array([[ -3.44924629e-05,  -9.23424959e-05,   1.06149353e-04, ...,
          3.55192460e-05,  -6.49478752e-05,   6.26562396e-05],
       [ -2.54213810e-05,  -6.56219199e-05,   7.18000811e-05, ...,
          2.56854109e-05,  -4.39891592e-05,   4.53768298e-05],
       [ -1.94022432e-05,  -6.50077127e-05,   7.06731807e-05, ...,
          2.21123919e-05,  -4.70438972e-05,   4.77186404e-05],
       ..., 
       [  3.25683504e-06,   1.99680217e-05,  -1.73030421e-05, ...,
         -5.45266084e-06,   9.08970833e-06,  -1.23563223e-05],
       [ -2.18548812e-05,  -6.81700767e-05,   7.66264275e-05, ...,
          2.75343191e-05,  -4.56795096e-05,   4.63980250e-05],
       [ -1.19311735e-05,  -2.92686746e-05,   3.22940759e-05, ...,
          1.04149804e-05,  -2.30316073e-05,   2.49901786e-05]], dtype=float32)

In [ ]:
sess.run(hidden_layer2)